# Python Database Live Coding

Goal of this live coding session is to use a local database to store data in a Python application on the Raspberry Pi.

## SQLite

In this workshow we will use a [SQLite](https://www.sqlite.org/index.html) database and the [`sqlite3`](https://docs.python.org/3/library/sqlite3.html) python library to store and retrieve data.

SQLite is a file-based *embedded database* written in C which is widely used by applications in almost every software aspect. All major web browsers, Android, iOS, Fedora, Lightroom, iTunes, BMW, ... either support SQLite or use a SQLite database internally. <sup>Sources [1](https://en.wikipedia.org/wiki/SQLite#Notable_uses), [2](https://www.sqlite.org/mostdeployed.html)</sup>

### Advantages

- easy to setup -> just use library and start
- serverless -> no configuration and little administration
- single file based database -> easy to migrate
- support by many programming languages
- small footprint < 500kB

### Disadvantages

- not fully SQL-92 standard

### [Quirks of SQLite](https://www.sqlite.org/quirks.html)

- "flexibly typed" -> `INSERT INTO Table (INTEGER_COLUMN) VALUES ('1234')`  is possible
- no boolean type -> `TRUE` and `FALSE` are represented as integers `1` and `0`, respectively  
*Note: 'TRUE', 'FALSE' can be used in SQLite but they are just aliases to `1` and `0`*
- no dedicated `DATETIME` type -> store as *ISO-8601* `TEXT` or as *Unix Time* `INTEGER`
- INTEGER != TEXT -> `SELECT 1='1';` returns `FALSE`

### Supported SQLite Types

SQLite supports the following types of data:

* NULL
* INTEGER
* REAL
* TEXT
* BLOB

### sqlite3 Library

`sqlite3` is the default python module to interact with SQlite databases and is part of the Python Standard Library and thus needs no explicit installation.

The following table shows the mapping between SQLite and Python types:


| SQLite    | Python    |
|-----------|-----------|
| NULL      | None      |
| INTEGER   | int       |
| REAL      | float     |
| TEXT      | str       |
| BLOB      | bytes     |


Since SQLite has no temporal types like `datetime` or `timestamp` we need to store them in either some string or integer represenation. Luckily, `sqlite3` provides default *adapters* and *convertes* for Python's `date` and `datetime` types. To store a `date` or `datetime` in a SQLite table, create the table with the column type `date` or `timestamp` for a Python `date` or `datetime` value:

```sql
CREATE TABLE Timetable(d date, ts timestamp)
```

## Create and Connect to Database

Calling `connect('db')` creates a connection to the 'db' database in the current working directory. If the database does not exists it get's created.

In [ ]:
import sqlite3
con = sqlite3.connect('accesspoint.db')

We see the newly created `accesspoint.db` in the current directory:

In [ ]:
%ls

### Calling SQL statements

Most SQL statements need a database cursor to work with SQLite. Use `cursor()` to get a cursor and `execute()` to execute SQL statemants against the connected database:

In [ ]:
cur = con.cursor()

# add IF NOT EXISTS to be able to execute cell multiple times
cur.execute("CREATE TABLE IF NOT EXISTS measurements(type INTEGER, value REAL, dateTimeUtc datetime)")

#### INSERT Part 1

Now we have a table `measurements` let's insert some dummy values with an `INSERT` statement.

Assume we got a temperature measurement of `20.23` degree celcius and an air humidity of `61.94%`.

Our measurement types are defined as follows:
- `1` = temperature
- `2` = air humidity

In [ ]:
cur.execute("""
    INSERT INTO measurements 
    VALUES  (1, 20.23, '2023-03-20 12:00:00.000000'),
            (2, 61.94, '2023-03-20 12:00:00.000000')
""")

# INSERTs need a commit to be written to the database
con.commit()

#### SELECT

After inserting some date, lets retrieve the data with a `SELECT` statement and take a closer look at the result.

In [ ]:
result = cur.execute("SELECT * FROM measurements")
result.fetchall()

We can also iterate over the result:

In [ ]:
result = cur.execute("SELECT * FROM measurements")

for result in result:
    print(result)

Or get only the one result of the query which can be useful for getting a max/min row if we know it returns only one row (`LIMIT 1` clause):

In [ ]:
result = cur.execute("SELECT * FROM measurements ORDER BY dateTimeUtc DESC LIMIT 1")
result.fetchone()

Checkout [VSCode plugin](#vscode-plugin---sqlite) for debugging.

#### INSERT Part 2

The first `INSERT` example assumed we have static data to insert, which is rarely the case *(maybe except for a initial setup script or seeding a database)*.

To insert data from Python we provide the data in the same format as we got the data in our `SELECT` statement above: `list` of `tuple`s

Since SQL statements are vulnverable to SQL injection attacks, we do not build the complete INSERT statement per hand with string formatting but use *placeholder* and *parameter substitution*.

The following two examples show the two kinds of placesholders, *question mark placeholders* and *named placeholders*, available in the `sqlite3` module.

##### Question Mark Placeholders

Question mark placeholders are used by replacing each parameter with a `?` in the SQL statement and provide the actual parameters as a `tuple` to the `execute` call.

Note: using `executemany()` allows to insert multiple rows by executing the given SQL statement for every item in the given list.

In [ ]:
from datetime import datetime, timedelta

now = datetime.utcnow()

print(f'Current time {now}')

# list of tuples
measurements = [
    (1, 21.84, now),
    (2, 55.10, now),
    (1, 21.01, now + timedelta(minutes=1)),
]

cur.executemany("INSERT INTO measurements VALUES  (?, ?, ?)", measurements)

# INSERTs need a commit to be written to the database
con.commit()

##### Named Placeholders

Named placeholders are used passing the data as a `dict` instead of `tuple` to the statement and replacing the values with the dict's key.

In [ ]:
from datetime import datetime, timedelta

now = datetime.utcnow()

print(f'Current time {now}')

measurements = [
    {"type": 1, "value": 20.87, "ts": now},
    {"type": 2, "value": 59.31, "ts": now},
    {"type": 1, "value": 20.13, "ts": now + timedelta(minutes=1)},
]

cur.executemany("INSERT INTO measurements VALUES  (:type, :value, :ts)", measurements)

# INSERTs need a commit to be written to the database
con.commit()


### Custom Adapters and Converters

If we have a Python data type which we want to insert as a single value in a table, we need to write our own custom `sqlite3` adapters and converters.

#### Adapter

An **adapter** let us convert a custom Python type to a SQLite value.

#### Converters
A **converter**  is used to convert a SQLite value to a custom Python type.  
Converters always receive a `bytes` object, regardless of the SQLite.

In most cases we will need to implement both.

Assume we add a `UUID` property to our measurements which uniquely identifies a measurement.

In the following example we will store the UUID as a string of hex digits. Since we receive `bytes` from the converter, we need to `decode()` them to get the string representation back.

Another option would be to store the UUID in it's byte representation. See [this](https://9to5answer.com/proper-way-to-store-guid-in-sqlite) example.

In [ ]:
import uuid

def uuid_adapter(uuid: uuid.UUID):
    return str(uuid)

def uuid_converter(bytes: bytes):
    return uuid.UUID(bytes.decode())


sqlite3.register_adapter(uuid.UUID, uuid_adapter)
sqlite3.register_converter('GUID', uuid_converter)

# or more compact as lambdas
sqlite3.register_adapter(uuid.UUID, lambda u: str(u))
sqlite3.register_converter('GUID', lambda s: uuid.UUID(s.decode()))

Before we can use the newly registered adapter and converter we need to reconnecto to the database and tell sqlite to parse the declared custom types with `detect_types=sqlite3.PARSE_DECLTYPES`.

In [ ]:
con.close()
con = sqlite3.connect('accesspoint.db', detect_types=sqlite3.PARSE_DECLTYPES)
cur = con.cursor()


To add the UUID column to our measurements table we will use the `ALTER TABLE` statement.

In [ ]:
cur.execute('ALTER TABLE measurements ADD COLUMN id GUID')

Now we can insert and retrieve data with an uuid type.

In [ ]:
from datetime import datetime

now = datetime.utcnow()

print(f'Current time {now}')

# list of tuples
measurements = [
    (1, 21.84, now, uuid.uuid4()),
    (2, 55.10, now, uuid.uuid4()),
]

# inserted measurement with UUID type
print(measurements[0])
print(type(measurements[0][3]))

cur.executemany("INSERT INTO measurements VALUES  (?, ?, ?, ?)", measurements)

# INSERTs need a commit to be written to the database
con.commit()

In [ ]:
result = cur.execute('SELECT * FROM measurements WHERE id NOT NULL ORDER BY dateTimeUtc DESC')
row = result.fetchone()

print(row)

# type of the retrieved data is UUID
print(type(row[3]))

# Live Coding

## Task 1

Try these samples on your Raspberry Pi to get familiar with the SQLite database and `sqlite3` module.

Time: ~5 mins


## Task 2

Write two small python scripts that read and write a SQLite database.

**Script `gen_measurements.py`**

The script `gen_measurements.py` should generate every few seconds a random measurement and write it into the `measurements` table in the `accesspoint.db` database.

**Script `get_measurements.py`**

The script `get_measurements.py` should ask the user for a measurement type (`1` for temperature, `2` for humidity) and return the highest/lowest measurement (with timestamps) and the average value of the last 1 minute.


Time: ~20 mins

### Helpful functions

#### `sleep()`

In [ ]:
import time

print('good night')
time.sleep(2)
print('hello')

#### `random()`

In [ ]:
import random

# random integer x with 10 <= x < 20 (similar to slicing/ranges)
print(random.randrange(10, 20))

# random integer x with 10 <= x <= 20
print(random.randint(10, 20))

#### `input()`

In [ ]:
value = input('Enter value: ')
print(f'Value is {value}')

# Appendix

## VSCode plugin - SQLite

VSCode has a SQLite plugin called [`SQLite`](https://marketplace.visualstudio.com/items?itemName=alexcvzz.vscode-sqlite) to explore and query SQLite databases.

It makes debugging a SQLite database much easier.

**NOTE:** Afer installing the plugin I needed to install sqlite3 on my raspberry (`sudo apt install sqlite3`) and reconnect VSCode to the remote raspberry (close and reopen).